In [16]:
import pandas as pd
import skcriteria as skc
import numpy as np
import matplotlib.pyplot as plt
import json
import yaml
import pprint
from tabulate import tabulate

from skcriteria.preprocessing import invert_objectives, scalers
from skcriteria.madm import simple

def get_id(key):
    criterion_key = {
        'Name' : 'name',
        'Elastic Modulus' : 'elastic_mod',
        'Yield Strength' : 'yield_strength',
        'Cost' : 'cost',
        'Ultimate Strength': 'ult_strength',
        'Machineability' : 'machineability'
    }

    if type(key) == list:
        result = [criterion_key.get(k) for k in key]
        result = [x for x in result if x != None]
        return result

    return criterion_key.get(key)

with open('C:/Users/ttrol/CodingProjects/MatNet/webscraper/results_files/AISI_steels_fakedata.yaml','r') as stream:
    try:
        raw_data = yaml.safe_load(stream)
    except FileNotFoundError:
        raise FileNotFoundError

raw_dataframe = pd.DataFrame(raw_data)


In [17]:
criterions = ["Elastic Modulus",
                       "Yield Strength", "Cost", "Ultimate Strength", "Machineability"]
raw_weights = [0, 0, 0, 0, -2]

formData = {}
for i in range(5):
    if criterions[i] != '' and raw_weights[i] != 0:
        formData[criterions[i]] = raw_weights[i]  

# print(formData)

weights = np.asarray(list(formData.values()))
weights = np.divide(weights,10)


objectives = []
for weight in weights:
    if weight < 0:
        objectives.append(min)
    elif weight > 0:
        objectives.append(max)


criterion_ids = get_id(list(formData.keys()))

df = raw_dataframe.loc[:, criterion_ids]        #type: ignore
# # let's says the first two alternatives are
# # for maximization and the last one for minimization

# we use the built-in function as aliases
row_id =  list(raw_dataframe.loc[:, 'name'])
dm = skc.mkdm(
    df.to_numpy(),
    objectives= objectives,
    weights= weights,
    alternatives = row_id,
    criteria= list(formData.keys())
)
dm


,Machineability[▼-0.2]
"AISI E9310H Steel, normalized, machined to 25 mm (1 in.) rounds, pseudocarburized, 150°C (300°F) temper",49
"AISI 4027 Steel, normalized, 14 mm (0.55 in.) round",2
"AISI 1020 Steel, mock carburized at 915°C (1680°F) for 8 hours, 775°C (1430°F) reheat, water quenched, 175°C (350°F) temper, 100 mm (4 in.) round",36
"AISI 1095 Steel, oil quenched 800°C (1470°F), 480°C (900°F) temper, 100 mm (4 in.) round",14
"AISI 4620H Steel, direct quench from pot, carburized, oil quenched, 150°C (300°F) temper",71
...,...
"AISI 1060 Steel, oil quenched 845°C (1550°F), 480°C (900°F) temper, 100 mm (4 in.) round",57
"AISI Grade 18Ni (250) Maraging Steel, 3.18 mm thick sheet, solution annealed at 815°C (1500°F) for 30 minutes, air cooled, aged at 455°C (850°F) for 3 hours",80
"AISI 1340H Steel, oil quenched from 830°C (1525°F), 595°C (1100°F) temper, 50 mm (2 in.) round",11
"AISI 4820 Steel, mock carburized, reheated to 800°C (1470°F), 150°C (300°F) temper, 13 mm (0.5 in.) round",57


In [18]:
from skcriteria.preprocessing import invert_objectives, scalers
from skcriteria.madm import simple

#SKC handles maximization or minimization only best , so we invert the third row to be a maximization problem 
#maximization is preferable to minimizationm
inverter = invert_objectives.InvertMinimize()
dmt = inverter.transform(dm)

#normalizes values
scaler = scalers.SumScaler(target="both")
dmt = scaler.transform(dmt)

dec = simple.WeightedSumModel()
dec

rank = dec.evaluate(dmt)  # we use the tansformed version of the data

rank

c:\Users\ttrol\CodingProjects\MatNet\.conda\lib\site-packages\skcriteria\preprocessing\invert_objectives.py:127: RuntimeWarning: divide by zero encountered in divide
  inverted_values = 1.0 / inv_mtx[:, minimize_mask]
c:\Users\ttrol\CodingProjects\MatNet\.conda\lib\site-packages\skcriteria\preprocessing\scalers.py:372: RuntimeWarning: invalid value encountered in divide
  return arr / sumval
c:\Users\ttrol\CodingProjects\MatNet\.conda\lib\site-packages\skcriteria\utils\rank.py:67: RuntimeWarning: invalid value encountered in cast
  return stats.rankdata(arr, "dense").astype(int)


ValueError: The data [-2147483648 -2147483648 -2147483648 ... -2147483648 -2147483648
 -2147483648] doesn't look like a ranking

In [ ]:
print(rank.e_.score)
print(rank.rank_)

df_with_score = raw_dataframe.assign(Score = rank.e_.score).assign(Rank = rank.rank_)
SortedDF = df_with_score.sort_values(by = 'Score', ascending = False)
SortedDF

[0.00036493 0.00067371 0.00089187 ... 0.00058233 0.0003041  0.00037428]
[239  97  43 ... 146 260 236]


,cost,elastic_mod,link,machineability,name,ult_strength,yield_strength,Score,Rank
1072,0.98,206,https://matweb.com/search/DataSheet.aspx?MatGU...,74,"AISI 1006 Steel, hot rolled bar, 19-32 mm (0.7...",295,165,0.001484,1
1266,7.76,200,https://matweb.com/search/DataSheet.aspx?MatGU...,8,"AISI 1008 Steel, hot rolled bar, 19-32 mm (0.7...",305,170,0.001436,2
11,0.84,200,https://matweb.com/search/DataSheet.aspx?MatGU...,11,"AISI 1010 Steel, hot rolled bar, 19-32 mm (0.7...",325,180,0.001347,3
1230,1.00,200,https://matweb.com/search/DataSheet.aspx?MatGU...,100,"AISI 1012 Steel, hot rolled bar, 19-32 mm (0.7...",330,185,0.001327,4
817,2.01,200,https://matweb.com/search/DataSheet.aspx?MatGU...,96,"AISI 1008 Steel, CQ, DQ, and DQSK sheet, 1.6-5...",330,210,0.001327,4
...,...,...,...,...,...,...,...,...,...
1138,9.77,200,https://matweb.com/search/DataSheet.aspx?MatGU...,23,"AISI Grade 18Ni (350) Maraging Steel, Aged, at...",2360,2320,0.000186,315
1046,4.04,203,https://matweb.com/search/DataSheet.aspx?MatGU...,21,"AISI Type A6 Tool Steel, austenitized 830-870°...",2380,2100,0.000184,316
49,5.54,200,https://matweb.com/search/DataSheet.aspx?MatGU...,50,"AISI Grade 18Ni (350) Maraging Steel, Aged, at...",2390,2350,0.000183,317
809,7.47,200,https://matweb.com/search/DataSheet.aspx?MatGU...,81,"AISI Grade 18Ni (350) Maraging Steel, Aged, at...",2420,2360,0.000181,318
